In [2]:
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose
from keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization
from keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from time import time
import numpy as np

# Configs

In [4]:
import keras.backend as K

In [3]:
N_CLASSES=1000
INPUT_SHAPE=224, 224, 3

# AlexNet

In [2]:
def alexnet(input_shape, n_classes):
    input = Input(input_shape)

    # actually batch normalization didn't exist back then
    # they used LRN (Local Response Normalization) for regularization
    x = Conv2D(96, 11, strides=4, padding='same', activation='relu')(input)
    x = BatchNormalization()(x)
    x = MaxPool2D(3, strides=2)(x)

    x = Conv2D(256, 5, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(3, strides=2)(x)

    x = Conv2D(384, 3, strides=1, padding='same', activation='relu')(x)

    x = Conv2D(384, 3, strides=1, padding='same', activation='relu')(x)

    x = Conv2D(256, 3, strides=1, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(3, strides=2)(x)

    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dense(4096, activation='relu')(x)

    output = Dense(n_classes, activation='softmax')(x)

    model = Model(input, output)
    return model

In [4]:
model = alexnet(INPUT_SHAPE, N_CLASSES)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 56, 56, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 56, 56, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 256)       0     

# VGG

In [5]:
def VGG(input_shape, n_classes):
    input = Input(input_shape)
    
    x = Conv2D(64, 3, padding='same', activation='relu')(input)
    x = Conv2D(64, 3, padding='same', activation='relu')(x)
    x = MaxPool2D(2, strides=2, padding='same')(x)
    
    x = Conv2D(128, 3, padding='same',activation='relu')(x)
    x = Conv2D(128, 3, padding='same',activation='relu')(x)
    x = MaxPool2D(2, strides=2, padding='same')(x)
    
    x = Conv2D(256, 3, padding='same',activation='relu')(x)
    x = Conv2D(256, 3, padding='same',activation='relu')(x)
    x = Conv2D(256, 3, padding='same',activation='relu')(x)
    x = MaxPool2D(2, strides=2, padding='same')(x)
    
    x = Conv2D(512, 3, padding='same',activation='relu')(x)
    x = Conv2D(512, 3, padding='same',activation='relu')(x)
    x = Conv2D(512, 3, padding='same',activation='relu')(x)
    x = MaxPool2D(2, strides=2, padding='same')(x)
    
    x = Conv2D(512, 3, padding='same',activation='relu')(x)
    x = Conv2D(512, 3, padding='same',activation='relu')(x)
    x = Conv2D(512, 3, padding='same',activation='relu')(x)
    x = MaxPool2D(2, strides=2, padding='same')(x)
    
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dense(4096, activation='relu')(x)
    output = Dense(n_classes, activation='softmax')(x)
    
    model = Model(input, output)
    return model

In [6]:
model = VGG(INPUT_SHAPE, N_CLASSES)

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 56, 56, 128)       0   

# Inception

In [8]:
def googlenet(input_shape, n_classes):
  
    def inception_block(x, f):
        t1 = Conv2D(f[0], 1, activation='relu')(x)

        t2 = Conv2D(f[1], 1, activation='relu')(x)
        t2 = Conv2D(f[2], 3, padding='same', activation='relu')(t2)

        t3 = Conv2D(f[3], 1, activation='relu')(x)
        t3 = Conv2D(f[4], 5, padding='same', activation='relu')(t3)

        t4 = MaxPool2D(3, 1, padding='same')(x)
        t4 = Conv2D(f[5], 1, activation='relu')(t4)

        output = Concatenate()([t1, t2, t3, t4])
        return output


    input = Input(input_shape)

    x = Conv2D(64, 7, strides=2, padding='same', activation='relu')(input)
    x = MaxPool2D(3, strides=2, padding='same')(x)

    x = Conv2D(64, 1, activation='relu')(x)
    x = Conv2D(192, 3, padding='same', activation='relu')(x)
    x = MaxPool2D(3, strides=2)(x)

    x = inception_block(x, [64, 96, 128, 16, 32, 32])
    x = inception_block(x, [128, 128, 192, 32, 96, 64])
    x = MaxPool2D(3, strides=2, padding='same')(x)

    x = inception_block(x, [192, 96, 208, 16, 48, 64])
    x = inception_block(x, [160, 112, 224, 24, 64, 64])
    x = inception_block(x, [128, 128, 256, 24, 64, 64])
    x = inception_block(x, [112, 144, 288, 32, 64, 64])
    x = inception_block(x, [256, 160, 320, 32, 128, 128])
    x = MaxPool2D(3, strides=2, padding='same')(x)

    x = inception_block(x, [256, 160, 320, 32, 128, 128])
    x = inception_block(x, [384, 192, 384, 48, 128, 128])

    x = AvgPool2D(7, strides=1)(x)
    x = Dropout(0.4)(x)

    x = Flatten()(x)
    output = Dense(n_classes, activation='softmax')(x)

    model = Model(input, output)
    return model

In [9]:
model = googlenet(INPUT_SHAPE, N_CLASSES)

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_18 (Conv2D)              (None, 112, 112, 64) 9472        input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_8 (MaxPooling2D)  (None, 56, 56, 64)   0           conv2d_18[0][0]                  
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, 56, 56, 64)   4160        max_pooling2d_8[0][0]            
____________________________________________________________________________________________

# Mobile Net

In [13]:
def mobilenet(input_shape, n_classes):
    
    def mobilenet_block(x, f, s=1):
        x = DepthwiseConv2D(3, strides=s, padding='same')(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)

        x = Conv2D(f, 1, strides=1, padding='same')(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        return x
    
    input = Input(input_shape)

    x = Conv2D(32, 3, strides=2, padding='same')(input)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = mobilenet_block(x, 64)
    x = mobilenet_block(x, 128, 2)
    x = mobilenet_block(x, 128)

    x = mobilenet_block(x, 256, 2)
    x = mobilenet_block(x, 256)

    x = mobilenet_block(x, 512, 2)
    
    for _ in range(5):
        x = mobilenet_block(x, 512)
    
    x = mobilenet_block(x, 1024, 2)
    x = mobilenet_block(x, 1024)

    x = GlobalAvgPool2D()(x)
    
    output = Dense(n_classes, activation='softmax')(x)
    
    model = Model(input, output)
    
    return model

In [14]:
model = mobilenet(INPUT_SHAPE, N_CLASSES)

model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_75 (Conv2D)           (None, 112, 112, 32)      896       
_________________________________________________________________
batch_normalization_3 (Batch (None, 112, 112, 32)      128       
_________________________________________________________________
re_lu (ReLU)                 (None, 112, 112, 32)      0         
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 112, 112, 32)      320       
_________________________________________________________________
batch_normalization_4 (Batch (None, 112, 112, 32)      128       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 112, 112, 32)      0   

# Shuffle Net

In [17]:
def shufflenet(input_shape, n_classes, g=8):
    channels = 384, 769, 1536
    repetitions = 3, 7, 3
    
    def ch_shuffle(x, g):
#     1 2 3 4 5 6 7 8 9 -reshape-> 1 2 3 -permute dims-> 1 4 7 -reshape-> 1 4 7 2 5 8 3 6 9
#                                  4 5 6                 2 5 8
#                                  7 8 9                 3 6 9
    
        _, w, h, ch = K.int_shape(x)
        ch_g = ch // g
    
        def shuffle_op(x):
            x = K.reshape(x, [-1, w, h, ch_g, g])
            x = K.permute_dimensions(x, [0, 1, 2, 4, 3])
            x = K.reshape(x, [-1, w, h, ch])
            return x

        x = Lambda(shuffle_op)(x)
        return x
    
    def gconv(tensor, ch, g):
        _, _, _, in_ch = K.int_shape(tensor)
        ch_g = in_ch // g
        out_ch = ch // g
        group = []

        for i in range(g):
            x = tensor[:, :, :, i*ch_g:(i+1)*ch_g]
            x = Lambda(lambda x: x[:, :, :, i*ch_g: (i+1)*ch_g])(tensor)
            x = Conv2D(out_ch, 1)(x)
            group.append(x)

        x = Concatenate()(group)
        return x
    
    def shufflenet_block(tensor, ch, s, g):
        x = gconv(tensor, ch // 4, g)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = ch_shuffle(x, g)
        x = DepthwiseConv2D(3, strides=s, padding='same')(x)
        x = BatchNormalization()(x)
        x = gconv(x, ch if s==1 else ch-K.int_shape(tensor)[-1], g)
        x = BatchNormalization()(x)

        if s == 1:
            x = Add()([tensor, x])
        else:
            avg = AvgPool2D(3, strides=2, padding='same')(tensor)
            x = Concatenate()([avg, x])

        output = ReLU()(x)
        return output
    
    def stage(x, ch, r, g):
        x = shufflenet_block(x, ch, 2, g)
        for i in range(r):
            x = shufflenet_block(x, ch, 1, g)
        return x
    
    input = Input(input_shape)
    x = Conv2D(24, 3, strides=2, padding='same')(input)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPool2D(3, strides=2, padding='same')(x)

    for ch, r in zip(channels, repetitions):
        x = stage(x, ch, r, g)

    x = GlobalAvgPool2D()(x)

    output = Dense(n_classes, activation='softmax')(x)

    model = Model(input, output)

    return model

In [18]:
model = shufflenet(INPUT_SHAPE, N_CLASSES)

model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_89 (Conv2D)              (None, 112, 112, 24) 672         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_30 (BatchNo (None, 112, 112, 24) 96          conv2d_89[0][0]                  
__________________________________________________________________________________________________
re_lu_27 (ReLU)                 (None, 112, 112, 24) 0           batch_normalization_30[0][0]     
____________________________________________________________________________________________

                                                                 conv2d_308[0][0]                 
                                                                 conv2d_309[0][0]                 
                                                                 conv2d_310[0][0]                 
                                                                 conv2d_311[0][0]                 
                                                                 conv2d_312[0][0]                 
                                                                 conv2d_313[0][0]                 
__________________________________________________________________________________________________
batch_normalization_72 (BatchNo (None, 7, 7, 1536)   6144        concatenate_39[0][0]             
__________________________________________________________________________________________________
add_10 (Add)                    (None, 7, 7, 1536)   0           re_lu_53[0][0]                   
          

# ResNet

In [19]:
def resnet(input_shape, n_classes):
  
    def conv_bn_rl(x, f, k=1, s=1, p='same'):
        x = Conv2D(f, k, strides=s, padding=p)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        return x
  
  
    def identity_block(tensor, f):
        x = conv_bn_rl(tensor, f)
        x = conv_bn_rl(x, f, 3)
        x = Conv2D(4*f, 1)(x)
        x = BatchNormalization()(x)

        x = Add()([x, tensor])
        output = ReLU()(x)
        return output
  
  
    def conv_block(tensor, f, s):
        x = conv_bn_rl(tensor, f)
        x = conv_bn_rl(x, f, 3, s)
        x = Conv2D(4*f, 1)(x)
        x = BatchNormalization()(x)

        shortcut = Conv2D(4*f, 1, strides=s)(tensor)
        shortcut = BatchNormalization()(shortcut)

        x = Add()([x, shortcut])
        output = ReLU()(x)
        return output
  
  
    def resnet_block(x, f, r, s=2):
        x = conv_block(x, f, s)
        for _ in range(r-1):
            x = identity_block(x, f)
        return x
    
  
    input = Input(input_shape)
  
    x = conv_bn_rl(input, 64, 7, 2)
    x = MaxPool2D(3, strides=2, padding='same')(x)

    x = resnet_block(x, 64, 3, 1)
    x = resnet_block(x, 128, 4)
    x = resnet_block(x, 256, 6)
    x = resnet_block(x, 512, 3)

    x = GlobalAvgPool2D()(x)

    output = Dense(n_classes, activation='softmax')(x)
  
    model = Model(input, output)
    return model

In [20]:
model = resnet(INPUT_SHAPE, N_CLASSES)

model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_346 (Conv2D)             (None, 112, 112, 64) 9472        input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_79 (BatchNo (None, 112, 112, 64) 256         conv2d_346[0][0]                 
__________________________________________________________________________________________________
re_lu_60 (ReLU)                 (None, 112, 112, 64) 0           batch_normalization_79[0][0]     
____________________________________________________________________________________________

# DenseNet

In [21]:
def densenet(img_shape, n_classes, f=32):
    repetitions = 6, 12, 24, 16
  
    def bn_rl_conv(x, f, k=1, s=1, p='same'):
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(f, k, strides=s, padding=p)(x)
        return x
  
  
    def dense_block(tensor, r):
        for _ in range(r):
            x = bn_rl_conv(tensor, 4*f)
            x = bn_rl_conv(x, f, 3)
            tensor = Concatenate()([tensor, x])
        return tensor
  
  
    def transition_block(x):
        x = bn_rl_conv(x, K.int_shape(x)[-1] // 2)
        x = AvgPool2D(2, strides=2, padding='same')(x)
        return x
  
  
    input = Input(img_shape)
  
    x = Conv2D(64, 7, strides=2, padding='same')(input)
    x = MaxPool2D(3, strides=2, padding='same')(x)
  
    for r in repetitions:
        d = dense_block(x, r)
        x = transition_block(d)
  
    x = GlobalAvgPool2D()(d)
  
    output = Dense(n_classes, activation='softmax')(x)
  
    model = Model(input, output)
    return model

In [22]:
model = densenet(INPUT_SHAPE, N_CLASSES)

model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_399 (Conv2D)             (None, 112, 112, 64) 9472        input_7[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_23 (MaxPooling2D) (None, 56, 56, 64)   0           conv2d_399[0][0]                 
__________________________________________________________________________________________________
batch_normalization_132 (BatchN (None, 56, 56, 64)   256         max_pooling2d_23[0][0]           
____________________________________________________________________________________________

Total params: 8,068,392
Trainable params: 7,986,920
Non-trainable params: 81,472
__________________________________________________________________________________________________


# Xception

In [25]:
def xception(input_shape, n_classes):
    
    def conv_bn(x, f, k, s=1, p='same'):
        x = Conv2D(f, k, strides=s, padding=p, use_bias=False)(x)
        x = BatchNormalization()(x)
        return x
    
    def sep_bn(x, f, k, s=1, p='same'):
        x = SeparableConv2D(f, k, strides=s, padding=p, use_bias=False)(x)
        x = BatchNormalization()(x)
        return x
    
    def entry_flow(x):
        x = conv_bn(x, 32, 3, 2)
        x = ReLU()(x)
        x = conv_bn(x, 64, 3)
        tensor = ReLU()(x)

        x = sep_bn(tensor, 128, 3)
        x = ReLU()(x)
        x = sep_bn(x, 128, 3)
        x = MaxPool2D(3, strides=2, padding='same')(x)

        tensor = conv_bn(tensor, 128, 1, 2)

        x = Add()([tensor, x])
        x = ReLU()(x)
        x = sep_bn(x, 256, 3)
        x = ReLU()(x)
        x = sep_bn(x, 256, 3)
        x = MaxPool2D(3, strides=2, padding='same')(x)

        tensor = conv_bn(tensor, 256, 1, 2)

        x = Add()([tensor, x])
        x = ReLU()(x)
        x = sep_bn(x, 728, 3)
        x = ReLU()(x)
        x = sep_bn(x, 728, 3)
        x = MaxPool2D(3, strides=2, padding='same')(x)

        tensor = conv_bn(tensor, 728, 1, 2)
        x = Add()([tensor, x])

        return x
    
    def middle_flow(tensor):
        for _ in range(8):
            x = ReLU()(tensor)
            x = sep_bn(x, 728, 3)
            x = ReLU()(x)
            x = sep_bn(x, 728, 3)
            x = ReLU()(x)
            x = sep_bn(x, 728, 3)

            tensor = Add()([tensor, x])
    
        return tensor

    def exit_flow(tensor):
        x = ReLU()(tensor)
        x = sep_bn(x, 728, 3)
        x = ReLU()(x)
        x = sep_bn(x, 1024, 3)
        x = MaxPool2D(3, strides=2, padding='same')(x)

        tensor = conv_bn(tensor, 1024, 1, 2)

        x = Add()([tensor, x])
        x = sep_bn(x, 1536, 3)
        x = ReLU()(x)
        x = sep_bn(x, 2048, 3)
        x = ReLU()(x)
        x = GlobalAvgPool2D()(x)
        x = Dense(n_classes, activation='softmax')(x)

        return x
    
    input = Input(input_shape)
    
    x = entry_flow(input)
    x = middle_flow(x)
    output = exit_flow(x)
    
    model = Model(input, output)
    
    return model

In [26]:
model = xception(INPUT_SHAPE, N_CLASSES)

model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_520 (Conv2D)             (None, 112, 112, 32) 864         input_8[0][0]                    
__________________________________________________________________________________________________
batch_normalization_252 (BatchN (None, 112, 112, 32) 128         conv2d_520[0][0]                 
__________________________________________________________________________________________________
re_lu_229 (ReLU)                (None, 112, 112, 32) 0           batch_normalization_252[0][0]    
____________________________________________________________________________________________

# Unet

In [27]:
def unet(input_shape=(572, 572, 1), f=64, steps=4, n_classes=2):
    
    def downstream(x, f):
        x = Conv2D(f, 3, activation='relu')(x)
        d = Conv2D(f, 3, activation='relu')(x)
        x = MaxPool2D(2, strides=2, padding='same')(d)
        return d, x
    
    def crop_merge(x, d):
        _, xw, xh, _ = K.int_shape(x)
        _, dw, dh, _ = K.int_shape(d)
        mw, mh = (dw-xw)//2, (dh-xh)//2

        d = Lambda(lambda x: x[:, mw: dw-mw, mh: dh-mh, :])(d)
        x = Concatenate()([d, x])
        return x
    
    def upstream(x, f, d):
        x = UpSampling2D()(x)
        x = Conv2D(f, 2, padding='same')(x)
        x = crop_merge(x, d)
        x = Conv2D(f, 3, activation='relu')(x)
        x = Conv2D(f, 3, activation='relu')(x)
        return x
    
    input = Input(input_shape)
    x = input
    
    downsampled = []
    for i in range(steps+1):
        d, x = downstream(x, f*2**i)
        downsampled.append(d)
    x = downsampled.pop()
    
    for i in range(steps-1, -1, -1):
        x = upstream(x, f*2**i, downsampled[i])
    
    output = Conv2D(n_classes, 1)(x)
    model = Model(input, output)
    
    return model

#lol...don't run this model again... it's angry...hahahaa!

# YOLO

In [1]:
def yolo(input_shape=(448, 448, 3), n_outputs=30):
    activation = LeakyReLU(0.1)

    def conv_1_3(x, f1, f2, r=1):
        for _ in range(r):
            x = Conv2D(f1, 1, padding='same', activation=activation)(x)
            x = Conv2D(f2, 3, padding='same', activation=activation)(x)
        return x
    
    input = Input(input_shape)

    x = Conv2D(64, 7, strides=2, padding='same', activation=activation)(input)
    x = MaxPool2D(2, strides=2, padding='same')(x)

    x = Conv2D(192, 3, padding='same', activation=activation)(x)
    x = MaxPool2D(2, strides=2, padding='same')(x)

    x = conv_1_3(x, 128, 256)
    x = conv_1_3(x, 256, 512)
    x = MaxPool2D(2, strides=2, padding='same')(x)

    x = conv_1_3(x, 256, 512, 4)
    x = conv_1_3(x, 512, 1024)
    x = MaxPool2D(2, strides=2, padding='same')(x)

    x = conv_1_3(x, 512, 1024, 2)
    x = Conv2D(1024, 3, padding='same', activation=activation)(x)
    x = Conv2D(1024, 3, strides=2, padding='same', activation=activation)(x)

    x = Conv2D(1024, 3, padding='same', activation=activation)(x)
    x = Conv2D(1024, 3, padding='same', activation=activation)(x)

    x = Dense(4096, activation=activation)(x)
    output = Dense(n_outputs)(x)
    
    model = Model(input, output)
    return model

In [5]:
K.clear_session()

In [6]:
model = yolo(INPUT_SHAPE, N_CLASSES)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 112, 112, 64)      9472      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 192)       110784    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 28, 192)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 128)       24704     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 256)       295168

# Yet to implement

- SqueezeNet
- RefineNet
- V-Net

# Models i had issues compiling on my PC

- UNET

-> literally frooze my PC twice... yet to investigate why...